[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-diffusion-diffusers-colab/blob/main/pytorch_stable_diffusion_3.ipynb)

In [ ]:
#@title Install - Step 1
!pip install -q transformers accelerate omegaconf xformers==0.0.20 opencv-contrib-python controlnet_aux
!pip install -q git+https://github.com/huggingface/diffusers

# !apt -y install -qq aria2
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_refiner_1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors -d /content/models -o sd_xl_refiner_1.0.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/juggernaut-xl/resolve/main/juggernautXL_version2.safetensors -d /content/models -o juggernautXL_version2.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M 'https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth' -d /content/models/controlnet -o control_v11p_sd15_canny.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M 'https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors' -d /content/models/controlnet -o control_v11p_sd15_canny_fp16.safetensors

In [ ]:
#@title Install - Step 2
import random, gc, torch

def image_grid(imgs, rows, cols):
    w,h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    for i, img in enumerate(imgs): grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
#@title Vanilla - Step 1
from diffusers import StableDiffusionXLPipeline
# pipeline = StableDiffusionXLPipeline.from_single_file("/content/models/juggernautXL_version2.safetensors", torch_dtype=torch.float16, use_safetensors=True, safety_checker=None ).to("cuda")
pipeline = StableDiffusionXLPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True, safety_checker=None ).to("cuda")
pipeline.enable_xformers_memory_efficient_attention()

In [ ]:
#@title Vanilla - Step 2
image = pipeline(prompt="a photo of a dog in a bucket", negative_prompt="blurry", num_inference_steps=50, height=1024, width=1024, guidance_scale=7).images[0]
image

In [ ]:
#@title ControlNet #Canny - Step 1
import torch
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel
# controlnet = ControlNetModel.from_single_file('/content/models/controlnet/control_v11p_sd15_canny.pth', torch_dtype=torch.float16) # invalid load key, '<'.
# controlnet = ControlNetModel.from_single_file('https://huggingface.co/lllyasviel/ControlNet-v1-1/blob/main/control_v11p_sd15_canny.pth', torch_dtype=torch.float16).to("cuda")
controlnet = ControlNetModel.from_pretrained("diffusers/controlnet-canny-sdxl-1.0", variant="fp16", torch_dtype=torch.float16).to("cuda")
pipeline = StableDiffusionXLControlNetPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", controlnet=controlnet, torch_dtype=torch.float16, variant="fp16", use_safetensors=True).to("cuda")
# pipeline.enable_model_cpu_offload()
pipeline.enable_xformers_memory_efficient_attention()

In [ ]:
#@title ControlNet #Canny - Step 2
from diffusers import UniPCMultistepScheduler
pipeline.scheduler = UniPCMultistepScheduler.from_config(pipeline.scheduler.config)

In [ ]:
#@title ControlNet #Canny - Step 3
from diffusers.utils import load_image
image = load_image("https://hf.co/datasets/huggingface/documentation-images/resolve/main/diffusers/input_image_vermeer.png")
image_raw = image

import cv2
from PIL import Image
import numpy as np
image = np.array(image)
low_threshold = 100
high_threshold = 200
image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)
image = image_grid([image_raw, canny_image], 1, 2)
image

In [ ]:
#@title ControlNet #Canny - Step 4
controlnet_conditioning_scale = 0.5
image = pipeline(prompt="a photo of a dog in a bucket", controlnet_conditioning_scale=controlnet_conditioning_scale, image=canny_image, negative_prompt="blurry", num_inference_steps=50, height=1024, width=1024, guidance_scale=7).images[0]
image

In [ ]:
#@title ControlNet #OpenPose - Step 1
import torch
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel
controlnet = ControlNetModel.from_pretrained("fusing/stable-diffusion-v1-5-controlnet-openpose", torch_dtype=torch.float16).to("cuda")
pipeline = StableDiffusionXLControlNetPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", controlnet=controlnet, torch_dtype=torch.float16, variant="fp16", use_safetensors=True).to("cuda")
# pipeline.enable_model_cpu_offload()
pipeline.enable_xformers_memory_efficient_attention()

In [ ]:
#@title ControlNet #Canny - Step 2
from diffusers import UniPCMultistepScheduler
pipeline.scheduler = UniPCMultistepScheduler.from_config(pipeline.scheduler.config)

In [ ]:
#@title ControlNet #OpenPose - Step 3
from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image
image = load_image("https://hf.co/datasets/YiYiXu/controlnet-testing/resolve/main/yoga1.jpeg")
image_raw = image
model = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")
pose = model(image)
image = image_grid([image_raw, pose], 1, 2)
image

In [ ]:
#@title ControlNet #OpenPose - Step 4
controlnet_conditioning_scale = 0.5
image = pipeline(prompt="a photo of a dog in a bucket", controlnet_conditioning_scale=controlnet_conditioning_scale, image=pose, negative_prompt="blurry", num_inference_steps=50, height=1024, width=1024, guidance_scale=7).images[0]
image